In [ ]:
import requests
import json
import os
from dotenv import load_dotenv
from sqlalchemy import create_engine, text
from sqlalchemy.exc import SQLAlchemyError

# Load credentials from .env
load_dotenv()

# Database connection info
DB_HOST = os.getenv("DB_HOST")
DB_PORT = os.getenv("DB_PORT")
DB_NAME = os.getenv("DB_NAME")
DB_USER = os.getenv("DB_USER")
DB_PASSWORD = os.getenv("DB_PASSWORD")
API_URL = "https://jobicy.com/api/v2/remote-jobs?count=5&geo=usa&industry=accounting-finance"

# Create SQLAlchemy engine
engine = create_engine(
    f"postgresql+psycopg2://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
)

# Fetch job data from API
response = requests.get(API_URL)
data = response.json()
jobs = data.get("jobs", [])



In [ ]:


# SQL statements
CREATE_TABLE_QUERY = """
CREATE TABLE IF NOT EXISTS remote_jobs (
    id INTEGER PRIMARY KEY,
    url TEXT,
    job_slug TEXT,
    job_title TEXT,
    company_name TEXT,
    company_logo TEXT,
    job_industry TEXT[],
    job_type TEXT[],
    job_geo TEXT,
    job_level TEXT,
    job_excerpt TEXT,
    job_description TEXT,
    pub_date TIMESTAMP,
    annual_salary_min INTEGER,
    annual_salary_max INTEGER,
    salary_currency TEXT
);
"""

INSERT_QUERY = """
INSERT INTO remote_jobs (
    id, url, job_slug, job_title, company_name, company_logo,
    job_industry, job_type, job_geo, job_level,
    job_excerpt, job_description, pub_date,
    annual_salary_min, annual_salary_max, salary_currency
)
VALUES (
    :id, :url, :jobSlug, :jobTitle, :companyName, :companyLogo,
    :jobIndustry, :jobType, :jobGeo, :jobLevel,
    :jobExcerpt, :jobDescription, :pubDate,
    :annualSalaryMin, :annualSalaryMax, :salaryCurrency
)
ON CONFLICT (id) DO NOTHING;
"""

# Execute DB operations
try:
    with engine.begin() as conn:  # automatically commits or rolls back
        conn.execute(text(CREATE_TABLE_QUERY))
        
        for job in jobs:
            job["jobIndustry"] = job.get("jobIndustry", [])
            job["jobType"] = job.get("jobType", [])
            job["jobDescription"] = job.get("jobDescription", "").strip()
            job["jobExcerpt"] = job.get("jobExcerpt", "").strip()
            
            conn.execute(text(INSERT_QUERY), job)

    print(f"{len(jobs)} job(s) inserted successfully.")

except SQLAlchemyError as e:
    print("Database error:", e)
